<a href="https://colab.research.google.com/github/FaizFP/Dicoding--Analystic-Sentiments/blob/main/Pelatihan_SentimenAnalys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [109]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE


In [100]:
nama_file = '/content/sample_data/dataset_final.csv'
df_bersih = pd.read_csv(nama_file)
print(df_bersih.head())

                                       clean_content sentimen
0  aplikasi bantu order gojek berangkat pulang kerja  positif
1  drivernya ambil pas hujan ga kaya grab yg maha...  positif
2                                  wow uda tahin aja  positif
3                                               good  positif
4  kecewa bgt ga segercep grab pake gojek kapok b...  negatif


In [101]:
df_bersih.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10692 entries, 0 to 10691
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   clean_content  10692 non-null  object
 1   sentimen       10692 non-null  object
dtypes: object(2)
memory usage: 167.2+ KB


In [102]:
df_bersih['clean_content'].isnull().sum()

0

In [108]:
df_bersih.duplicated().sum()

3415

In [103]:

# 1. Inisialisasi TfidfVectorizer
# max_features=5000 berarti kita hanya akan mengambil 5000 kata/fitur yang paling penting
# Ini membantu model fokus pada kata-kata yang paling relevan dan mempercepat training
vectorizer = TfidfVectorizer(max_features=5000)

# 2. Definisikan variabel fitur (X) dan target (y)
# Fitur (X) adalah teks ulasan yang sudah bersih
# Target (y) adalah label sentimen yang sudah kita buat
X_text = df_bersih['clean_content']
y = df_bersih['sentimen']

# 3. Lakukan proses fit_transform
# 'fit' mempelajari kosakata dari data teks Anda
# 'transform' mengubah teks tersebut menjadi matriks angka TF-IDF
print("Memulai proses ekstraksi fitur dengan TF-IDF...")
X = vectorizer.fit_transform(X_text)
print("Ekstraksi fitur selesai.")

# 4. Periksa hasil dari proses ini
print(f"\nDimensi dari matriks fitur (X): {X.shape}")
print(f"Dimensi dari target (y): {y.shape}")

Memulai proses ekstraksi fitur dengan TF-IDF...
Ekstraksi fitur selesai.

Dimensi dari matriks fitur (X): (10692, 5000)
Dimensi dari target (y): (10692,)


## 🧪 Skema 1: SVM + TF-IDF (Data 80/20)


In [104]:
# 1. Pembagian Data (80% Latih, 20% Uji)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# 2. Pelatihan Model SVM
print("Memulai pelatihan model SVM...")
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)
print("Pelatihan SVM selesai.")

# 3. Evaluasi Model
y_pred_svm = svm_model.predict(X_test)
accuracy_svm = accuracy_score(y_test, y_pred_svm)

print(f"\n--- Hasil Skema 1: SVM + TF-IDF (80/20) ---")
print(f"Akurasi: {accuracy_svm * 100:.2f}%")
print("\nLaporan Klasifikasi:")
print(classification_report(y_test, y_pred_svm))

Memulai pelatihan model SVM...
Pelatihan SVM selesai.

--- Hasil Skema 1: SVM + TF-IDF (80/20) ---
Akurasi: 88.17%

Laporan Klasifikasi:
              precision    recall  f1-score   support

     negatif       0.79      0.82      0.80       583
      netral       0.00      0.00      0.00        81
     positif       0.92      0.95      0.94      1475

    accuracy                           0.88      2139
   macro avg       0.57      0.59      0.58      2139
weighted avg       0.85      0.88      0.86      2139



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Tuning 2

In [105]:

# Inisialisasi model SVM dengan parameter terbaik DAN class_weight
svm_model_balanced = SVC(
    kernel='linear',
    C=1,  # C terbaik dari hasil GridSearchCV Anda
    class_weight='balanced', # <-- Tambahkan parameter penting ini
    random_state=42
)

print("Memulai pelatihan model SVM dengan class_weight='balanced'...")
svm_model_balanced.fit(X_train, y_train)
print("Pelatihan selesai.")

# Evaluasi Model
y_pred_balanced = svm_model_balanced.predict(X_test)
accuracy_balanced = accuracy_score(y_test, y_pred_balanced)

print(f"\n--- Hasil SVM dengan Class Weight Balanced ---")
print(f"Akurasi: {accuracy_balanced * 100:.2f}%")
print("\nLaporan Klasifikasi:")
print(classification_report(y_test, y_pred_balanced))

Memulai pelatihan model SVM dengan class_weight='balanced'...
Pelatihan selesai.

--- Hasil SVM dengan Class Weight Balanced ---
Akurasi: 84.57%

Laporan Klasifikasi:
              precision    recall  f1-score   support

     negatif       0.75      0.77      0.76       583
      netral       0.14      0.23      0.17        81
     positif       0.96      0.91      0.93      1475

    accuracy                           0.85      2139
   macro avg       0.62      0.64      0.62      2139
weighted avg       0.87      0.85      0.86      2139



Tuning

In [106]:
param_grid_expanded = [
    {
        'kernel': ['linear'],
        'C': [1, 10, 100]
    },
    {
        'kernel': ['rbf'],
        'C': [1, 10, 100],
        'gamma': [0.1, 0.01, 0.001]
    }
]

# 2. Inisialisasi GridSearchCV dengan param_grid yang baru
# Perhatikan kita tidak lagi mendefinisikan kernel di dalam SVC()
grid_search_expanded = GridSearchCV(
    SVC(random_state=42), # Model dasar tanpa kernel spesifik
    param_grid_expanded,  # Gunakan grid parameter yang sudah diperluas
    cv=3,                 # cv=3 untuk mempercepat pencarian yang lebih kompleks ini
    n_jobs=-1,
    verbose=2
)

# 3. Jalankan pencarian pada data latih Anda (X_train dan y_train)
print("Memulai pencarian hyperparameter yang diperluas (kernel, C, gamma)...")
grid_search_expanded.fit(X_train, y_train)
print("Pencarian selesai.")

# 4. Lihat kombinasi parameter terbaik yang ditemukan
print(f"\nParameter terbaik yang ditemukan: {grid_search_expanded.best_params_}")
print(f"Skor cross-validation terbaik: {grid_search_expanded.best_score_:.4f}")


# 5. Gunakan model terbaik untuk membuat prediksi
best_model_overall = grid_search_expanded.best_estimator_
y_pred_best = best_model_overall.predict(X_test)
accuracy_best = accuracy_score(y_test, y_pred_best)

# 6. Tampilkan hasil akurasi yang sudah ditingkatkan
print(f"\n--- Hasil Terbaik Setelah Pencarian Diperluas ---")
print(f"Akurasi: {accuracy_best * 100:.2f}%")
print("\nLaporan Klasifikasi:")
print(classification_report(y_test, y_pred_best))

Memulai pencarian hyperparameter yang diperluas (kernel, C, gamma)...
Fitting 3 folds for each of 12 candidates, totalling 36 fits
Pencarian selesai.

Parameter terbaik yang ditemukan: {'C': 1, 'kernel': 'linear'}
Skor cross-validation terbaik: 0.8741

--- Hasil Terbaik Setelah Pencarian Diperluas ---
Akurasi: 88.17%

Laporan Klasifikasi:
              precision    recall  f1-score   support

     negatif       0.79      0.82      0.80       583
      netral       0.00      0.00      0.00        81
     positif       0.92      0.95      0.94      1475

    accuracy                           0.88      2139
   macro avg       0.57      0.59      0.58      2139
weighted avg       0.85      0.88      0.86      2139



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [107]:
```# 1. Definisikan parameter yang ingin diuji
# Kita akan mencoba beberapa nilai C yang umum digunakan
param_grid = {
    'C': [0.1, 1, 10, 100]
}

# 2. Inisialisasi GridSearchCV
# - SVC(kernel='linear', random_state=42): Model dasar kita
# - param_grid: 'Kamus' pengaturan yang akan diuji
# - cv=5: Menggunakan 5-fold cross-validation untuk evaluasi yang lebih stabil
# - n_jobs=-1: Menggunakan semua core CPU untuk mempercepat proses pencarian
grid_search = GridSearchCV(
    SVC(kernel='linear', random_state=42),
    param_grid,
    cv=5,
    n_jobs=-1,
    verbose=2 # Menampilkan log proses pencarian
)

# 3. Jalankan pencarian pada data latih Anda
print("Memulai pencarian hyperparameter terbaik untuk SVM...")
grid_search.fit(X_train, y_train)
print("Pencarian selesai.")

# 4. Lihat parameter terbaik yang ditemukan
print(f"\nParameter 'C' terbaik yang ditemukan: {grid_search.best_params_}")

# 5. Gunakan model terbaik untuk membuat prediksi
best_svm_model = grid_search.best_estimator_
y_pred_best_svm = best_svm_model.predict(X_test)
accuracy_best_svm = accuracy_score(y_test, y_pred_best_svm)

# 6. Tampilkan hasil akurasi yang sudah ditingkatkan
print(f"\n--- Hasil Skema 1 (Setelah Tuning) ---")
print(f"Akurasi: {accuracy_best_svm * 100:.2f}%")
print("\nLaporan Klasifikasi:")
print(classification_report(y_test, y_pred_best_svm))

SyntaxError: invalid syntax (ipython-input-3563197383.py, line 1)

## 🌳 Skema 2: Random Forest + Word2Vec (Data 80/20)


In [ ]:
!pip install gensim

# A. Tokenisasi data teks (Word2Vec butuh input berupa list of list of words)
tokenized_text = [text.split() for text in df_bersih['clean_content']]

# B. Latih model Word2Vec
# size: dimensi vektor, window: jarak kata, min_count: abaikan kata yg jarang muncul
print("Memulai pelatihan model Word2Vec...")
w2v_model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4)
print("Pelatihan Word2Vec selesai.")

# C. Ubah setiap ulasan menjadi satu vektor rata-rata
def vectorize_review(review, model):
    words = review.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv.key_to_index]
    if not word_vectors:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

X_w2v = np.array([vectorize_review(review, w2v_model) for review in df_bersih['clean_content']])
y_w2v = df_bersih['sentimen']

print(f"Dimensi matriks fitur Word2Vec (X_w2v): {X_w2v.shape}")

In [ ]:

# 1. Pembagian Data (80% Latih, 20% Uji)
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(
    X_w2v, y_w2v,
    test_size=0.2,
    random_state=42,
    stratify=y_w2v
)

# 2. Pelatihan Model Random Forest
print("Memulai pelatihan model Random Forest...")
rf_model_w2v = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_w2v.fit(X_train_w2v, y_train_w2v)
print("Pelatihan Random Forest selesai.")

# 3. Evaluasi Model
y_pred_rf_w2v = rf_model_w2v.predict(X_test_w2v)
accuracy_rf_w2v = accuracy_score(y_test_w2v, y_pred_rf_w2v)

print(f"\n--- Hasil Skema 2: RF + Word2Vec (80/20) ---")
print(f"Akurasi: {accuracy_rf_w2v * 100:.2f}%")
print("\nLaporan Klasifikasi:")
print(classification_report(y_test_w2v, y_pred_rf_w2v))

In [ ]:
# 1. Pembagian Data (70% Latih, 30% Uji)
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(
    X, y,
    test_size=0.3, # Perhatikan test_size diubah menjadi 0.3
    random_state=42,
    stratify=y
)

# 2. Pelatihan Model Random Forest
print("Memulai pelatihan model Random Forest...")
rf_model_tfidf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_tfidf.fit(X_train_rf, y_train_rf)
print("Pelatihan Random Forest selesai.")

# 3. Evaluasi Model
y_pred_rf_tfidf = rf_model_tfidf.predict(X_test_rf)
accuracy_rf_tfidf = accuracy_score(y_test_rf, y_pred_rf_tfidf)

print(f"\n--- Hasil Skema 3: RF + TF-IDF (70/30) ---")
print(f"Akurasi: {accuracy_rf_tfidf * 100:.2f}%")
print("\nLaporan Klasifikasi:")
print(classification_report(y_test_rf, y_pred_rf_tfidf))